# Week 4 Quiz

In [15]:
import os
import cv2
import numpy as np
from skimage import io
import matplotlib.pyplot as plt

All three of the quiz questions are concerning the transformation between the following two images:
quiz image 1.png and quiz image 2.png

In [16]:
I1 = io.imread(os.path.join('..', 'solutions', 'Data', 'week4', 'quiz_image_1.png'))
I2 = io.imread(os.path.join('..', 'solutions', 'Data', 'week4', 'quiz_image_2.png'))

In [17]:
def match_SIFT(im1, im2, thres = 0.6):
    
    # Initiate SIFT detector
    sift = cv2.SIFT_create()
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(im1,None)
    kp2, des2 = sift.detectAndCompute(im2,None)
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    
    # Apply ratio test
    good_matches = []
    
    for m,n in matches:
        if m.distance/(n.distance+10e-10) < thres:
            good_matches.append([m])
    
    # Find coordinates
    pts_im1 = [kp1[m[0].queryIdx].pt for m in good_matches]
    pts_im1 = np.array(pts_im1, dtype=np.float32).T
    pts_im2 = [kp2[m[0].trainIdx].pt for m in good_matches]
    pts_im2 = np.array(pts_im2, dtype=np.float32).T
    return pts_im1, pts_im2

In [18]:
p, q = match_SIFT(I1, I2)
m_p = np.mean(p, axis=1, keepdims=True)
m_q = np.mean(q, axis=1, keepdims=True)

In [19]:
s = np.sum(np.linalg.norm(p-m_p)) / np.sum(np.linalg.norm(q-m_q))
s

2.7079582

In [20]:
#Finding rotation matrix R
C = (p - m_p)@(q - m_q).T
C

array([[ -8594531. ,   4787780.5],
       [ -3771469.2, -11677374. ]], dtype=float32)

In [21]:
U, S, VT = np.linalg.svd(C)

In [22]:
R_hat = U@VT

In [23]:
D = np.array([
    [1, 0],
    [0, np.linalg.det(R_hat)]
])
R = R_hat@D

In [24]:
t = np.mean(q - s*R@p, axis=1, keepdims=True)
t

array([[ 991.4803809 ],
       [2066.04370613]])

## 1)
What is the length of the translation vector (in pixels) between original (not rotated) points?
That is, what is the distance between µq and µp?

In [26]:
np.linalg.norm(m_p-m_q)

472.4212

## 2)
What is the magnification scale (i.e. the larger of the numbers s and 1/s)?

In [30]:
np.max([s, 1/s])

2.707958221435547

## 3)
What is the absolute angle of the rotation in degrees?

In [31]:
R[0,0])

array([[-0.92124957,  0.38897207],
       [-0.38897207, -0.92124951]])

In [39]:
# computing angle theta from cos(theta)
np.arccos(R[0,0])/ np.pi*180

157.10944938236557